In [45]:
import io
import numpy as np
import pandas as pd
import requests
import statsmodels.api as sm

In [46]:
URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv"
df = pd.read_csv(io.BytesIO(requests.get(URL).content), sep=",", encoding="unicode_escape")

In [47]:
data = df[["Rented Bike Count", "Rainfall(mm)", "Temperature(°C)"]].to_numpy().astype(np.float64)
data = np.hstack((data, data[:, 1:] @ np.array([3, 2]).reshape(-1, 1))) # Some linear combination.
x = data[:, 1:]
y = data[:, 0]
desmat = np.hstack((x, np.ones((x.shape[0], 1))))

In [48]:
print(f"Design matrix size: {desmat.shape}")
print(f"Design matrix rank: {np.linalg.matrix_rank(desmat)}")
print(f"Correlation matrix of design matrix:\n{np.corrcoef(desmat.T)}")

Design matrix size: (8760, 4)
Design matrix rank: 3
Correlation matrix of design matrix:
[[1.         0.05028186 0.18874589        nan]
 [0.05028186 1.         0.99027426        nan]
 [0.18874589 0.99027426 1.                nan]
 [       nan        nan        nan        nan]]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [49]:
# Method 1.
beta_1 = np.linalg.inv(desmat.T @ desmat) @ desmat.T @ y
rsq_1 = np.corrcoef(desmat @ beta_1, y)[0, 1]**2

In [50]:
# Method 2.
beta_2 = np.linalg.lstsq(desmat, y, rcond=None)[0]
rsq_2 = np.corrcoef(desmat @ beta_2, y)[0, 1]**2

In [51]:
# Method 3.
model = sm.OLS(y, sm.add_constant(x)).fit()
beta_3 = model.params
rsq_3 = model.rsquared

In [52]:
print("R^2:")
print(f"\tleft-inverse       : {rsq_1:.6f}")
print(f"\tnp.linalg.lstsq    : {rsq_2:.6f}")
print(f"\tstatsmodels.api.OLS: {rsq_3:.6f}")

print("beta:")
print(f"\tleft-inverse       : {beta_1.T}")
print(f"\tnp.linalg.lstsq    : {beta_2.T}")
print(f"\tstatsmodels.api.OLS: {beta_3.T}")

R^2:
	left-inverse       : 0.264760
	np.linalg.lstsq    : 0.312648
	statsmodels.api.OLS: 0.312648
beta:
	left-inverse       : [ 12.72950902  10.3442932    7.96516844 337.48329388]
	np.linalg.lstsq    : [-43.37481759  57.94774471 -14.22896334 337.48329388]
	statsmodels.api.OLS: [337.48329388 -43.37481759  57.94774471 -14.22896334]
